In [3]:
import os
print("Current working directory:", os.getcwd())

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import sys # Path to Python interpreter
print(sys.executable)


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# Read local csv file
df = pd.read_csv("../data/ai_job_dataset.csv")

# Helper code to confirm code is running
print(df.head())
print(df.columns)

# Lines below not for local use
# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

Current working directory: c:\Users\blake\Desktop\AI Special Topics Final\AI-Job-Market-and-Salary-Trends\notebooks
c:\Python313\python.exe
    job_id              job_title  salary_usd salary_currency  \
0  AI00001  AI Research Scientist       90376             USD   
1  AI00002   AI Software Engineer       61895             USD   
2  AI00003          AI Specialist      152626             USD   
3  AI00004           NLP Engineer       80215             USD   
4  AI00005          AI Consultant       54624             EUR   

  experience_level employment_type company_location company_size  \
0               SE              CT            China            M   
1               EN              CT           Canada            M   
2               MI              FL      Switzerland            L   
3               SE              FL            India            M   
4               EN              PT           France            S   

  employee_residence  remote_ratio  \
0              China  